In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate

chat = ChatOpenAI(
    temperature=0.1
)
# temperature: 얼마나 창의적인가 0~1

template = PromptTemplate.from_template(
    "{country_a}과(와) {country_b} 사이의 거리는 어떻게 되나요?"
)

prompt = template.format(country_a = "한국", country_b= "일본")


chat.predict(prompt)

# from langchain.llms import OpenAI
# llm = OpenAI(model_name="gpt-3.5-turbo-0125")
# a = llm.predict("How many planets are there?")
# a

In [ ]:
# from langchain.schema import HumanMessage, AIMessage, SystemMessage

template = ChatPromptTemplate.from_messages([
    ("system", "You are a geography export. And you only reply in {language}."),
    ("ai", "안녕하세요. 제 이름은 {name}입니다."),
    ("human", "{country_a}과(와) {country_b} 사이의 거리는 어떻게 되나요?. 그리고 당신의 이름은 무엇인가요?"),
])

prompt = template.format_messages(
    language = "korean",
    name = "쫀떡",
    country_a = "한국",
    country_b = "일본",
)

chat.predict_messages(prompt)

In [ ]:
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items = text.strip().split(",")
        return list(map(str.strip, items))
    # strip: text 앞뒤 공백을 잘라준다. -> 텍스트ㅡ이 시작부분이나 끝부분이 공백이라면 제거
    
p = CommaOutputParser()
p.parse("Hello,   how,are,you")

In [ ]:
template = ChatPromptTemplate.from_messages([
    ("system", "당신은 list 생성 기계입니다. 질문들은 모두 최대 {max_items}개, comma로 구분해주세요. 그리고 영어로 답할 경우 소문자로만 답해주세요. list가 아닌 것으로 답을 하지마세요"),
    ("human", "{question}"),
    ])

prompt = template.format_messages(max_items=  "10", question = "What are the colors?")
result = chat.predict_messages(prompt)
p = CommaOutputParser()
p.parse(result.content)

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.schema import BaseOutputParser

chat = ChatOpenAI(
    temperature=0.1
)

class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items = text.strip().split(",")
        return list(map(str.strip, items))
    # strip: text 앞뒤 공백을 잘라준다. -> 텍스트ㅡ이 시작부분이나 끝부분이 공백이라면 제거
    
template = ChatPromptTemplate.from_messages([
    ("system", "당신은 list 생성 기계입니다. 질문들은 모두 최대 {max_items}개, comma로 구분해주세요. 그리고 영어로 답할 경우 소문자로만 답해주세요. list가 아닌 것으로 답을 하지마세요"),
    ("human", "{question}"),
    ])

prompt = template.format_messages(max_items=  "10", question = "What are the colors?")
result = chat.predict_messages(prompt)
p = CommaOutputParser()
p.parse(result.content)

# chat model, outputParser, template

In [ ]:
# chain: 모든 요소를 합쳐주는 역할을 한다.
# 합쳐진 요소들은 하나의 chain으로 실행된다. 

from langchain.chat_models import ChatOpenAI, ChatAnthropic
from langchain.prompts import ChatPromptTemplate
from langchain.schema import BaseOutputParser

# ChatModel
# chat = ChatAnthropic(temperature=0.1)
chat = ChatOpenAI(
    temperature=0.1
)


# OutputParser
class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items = text.strip().split(",")
        return list(map(str.strip, items))
    # strip: text 앞뒤 공백을 잘라준다. -> 텍스트ㅡ이 시작부분이나 끝부분이 공백이라면 제거
    
    
# Prompt
template = ChatPromptTemplate.from_messages([
    ("system", "당신은 list 생성 기계입니다. 질문들은 모두 최대 {max_items}개, comma로 구분해주세요. 그리고 영어로 답할 경우 소문자로만 답해주세요. list가 아닌 것으로 답을 하지마세요"),
    ("human", "{question}"),
    ])


chain_one = template | chat | CommaOutputParser()
# chain_two = template_2 | chat | outputparser
# all = chain_one | chain_two | outputparser 

chain_one.invoke({
    "max_items": 5,
    "question": "What are the pokemons?"
})

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat_model = ChatOpenAI(temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()])
# streaming: model의 응답이 생성되는 걸 볼 수 있게 해준다.

chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 세계적인 요리사입니다. 당신은 쉬운 제료로 쉽게 만들 수 있는 음식 제조법을 알려줘야 합니다."),
    ("human", "나는 {cuisine}을 만들고 싶습니다."),
])

chef_chain = chef_prompt | chat_model

In [ ]:
veg_chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a vegetarian chef specialized on making traditional recipes vegetarian. You find alternative ingredients and explain their preparation. You don't radically modify the recipe. If there is no alternative for a food just say you don't know how to recipe it."),
    ("human", "{recipe}"),
])

veg_chef_chain = veg_chef_prompt | chat_model

final_chain = {"recipe": chef_chain} | veg_chef_chain # veg_chain의 입력값 recipe를 chef_chain으로부터 뱓는다.
final_chain.invoke({
    "cuisine":"카레",
})
# chef_chain으로부터 얻은 결과가 recipe라는 key에 저장된다.

# 모듈
LangChain은 다음과 같은 주요 모듈을 위한 확장 가능한 표준 인터페이스 및 외부 통합 기능을 제공합니다:

- Model I/O
언어 모델과의 인터페이스

- Retrieval
애플리케이션별 데이터를 사용한 인터페이스

- Agents
높은 수준의 지침이 주어지면 체인이 어떤 도구를 사용할지 선택하도록 합니다

- Chains
일반적인 빌딩 블록 구성

- Memory
체인 실행 간 지속적인 응용 프로그램 상태

- Callbacks
모든 체인의 중간 단계 기록 및 스트리밍


In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()])

# t = PromptTemplate(
#     template="What is the capital of {country}",
#     input_variables = ["country"],
# )
# t = PromptTemplate.from_template("What is the capital of {country}")
# t.format(country="korea")

examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]


example_prompt = PromptTemplate.from_template("Human: {question}\nAI:{answer}")

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"],
)

chain = prompt | chat

chain.invoke({"country": "Turkey"})

AI:
        I know this:
        Capital: Ankara
        Language: Turkish
        Food: Kebab and Baklava
        Currency: Turkish Lira

AIMessageChunk(content='AI:\n        I know this:\n        Capital: Ankara\n        Language: Turkish\n        Food: Kebab and Baklava\n        Currency: Turkish Lira')

In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotPromptTemplate, FewShotChatMessagePromptTemplate
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat_model = ChatOpenAI(temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()])

examples = [
    {
        "country": "France",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "country": "Italy",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "country": "Greece",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

# example_template = """
#     Human: {question}
#     Ai: {answer}
# """
# example_prompt = PromptTemplate.from_template(example_template)
# example_prompt = PromptTemplate.from_template("Human: {question}\nAi: {answer}")

example_prompt = ChatPromptTemplate.from_messages([
    ("human", "{country}에 대해 알려줘"),
    ("ai", "{answer}")
])

# prompt = FewShotPromptTemplate(
example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
    # suffix="Human: What do you know about {country}?",
    # input_variables=["country"]
)

final_prompt = ChatPromptTemplate.from_messages([
    ("system", "너는 지리 전문가야"),
    example_prompt,
    ("human", "{country}에 대해 알려줘" ),
])

chain = final_prompt | chat_model
chain.invoke({
    "country": "Korea"
})

한국에 대해 알려드리겠습니다:
수도: 서울
언어: 한국어
음식: 김치, 불고기
통화: 대한민국 원

AIMessageChunk(content='한국에 대해 알려드리겠습니다:\n수도: 서울\n언어: 한국어\n음식: 김치, 불고기\n통화: 대한민국 원')

In [8]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
# from langchain.prompts.example_selector import LengthBasedExampleSelector

chat_model = ChatOpenAI(temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()])

examples = [
    {
        "country": "France",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "country": "Italy",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "country": "Greece",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

# example_template = """
#     Human: {question}
#     Ai: {answer}
# """

example_prompt = ChatPromptTemplate.from_messages([
    ("human", "{country}에 대해 알려줘"),
    ("ai", "{answer}")
])

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages([
    ("system", "너는 지리 전문가야"),
    example_prompt,
    ("human", "{c}에 대해 알려줘" ),
])

chain = final_prompt | chat_model
chain.invoke({
    "c": "Korea"
})

한국에 대해 알려드리겠습니다:
수도: 서울
언어: 한국어
음식: 김치, 불고기
통화: 대한민국 원

AIMessageChunk(content='한국에 대해 알려드리겠습니다:\n수도: 서울\n언어: 한국어\n음식: 김치, 불고기\n통화: 대한민국 원')

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.example_selector import LengthBasedExampleSelector
from langchain.prompts.example_selector.base import BaseExampleSelector


chat_model = ChatOpenAI(temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()])

examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

# 예제 선택기 
class RandomExampleSelect(BaseExampleSelector):
    
    def __init__(self, examples):
        self.examples = examples
    
    def add_example(self, example):
        self.examples.append(example)
    
    def select_examples(self, input_variables):
        from random import choice
        return [choice(self.examples)]

example_prompt = PromptTemplate.from_template("Human: {question}\nAi: {answer}")

example_selector = RandomExampleSelect(
    examples = examples,
)

example_prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,
    suffix = "Human: {country}에 대해 알려줘",
    input_variables = ["country"]
)

prompt.format(country="Korea")

In [ ]:
# serialization
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import load_prompt

# prompt = load_prompt("./prompt.json")
prompt = load_prompt("./prompt.yaml")

chat_model = ChatOpenAI(temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()])

prompt.format(country= "Korea")

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.pipeline import PipelinePromptTemplate

chat_model = ChatOpenAI(temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()])


intro = PromptTemplate.from_template(
    """
    당신은 역할놀이 보조자입니다. 그리고 당신은 {character} 역을 맡아야 합니다.
"""
)

example = PromptTemplate.from_template(
    """
    이건 어떻게 말해야 하는지 예시야:
    Human: {example_question}
    You: {example_answer}
"""
)

start = PromptTemplate.from_template(
        """
    시작!
    Human: {question}
    You:
"""
)

final = PromptTemplate.from_template(
        """
    {intro}
    
    {example}
    
    {start}
"""
)

prompts = [
    ("intro", intro),
    ("example", example),
    ("start", start)
]
full_prompt = PipelinePromptTemplate(final_prompt=final, pipeline_prompts=prompts)

# 확인
# full_prompt.format(
#     character = "해적",
#     example_question = "어디에 있어?",
#     example_answer = "크하하하! 그건 비밀이다!",
#     question = "너가 가장 좋아하는 음식은 뭐야?"
# )

chain = full_prompt | chat_model
chain.invoke({
    "character": "해적",
    "example_question": "어디에 있어?",
    "example_answer": "크하하하! 그건 비밀이다!",
    "question": "너가 가장 좋아하는 음식은 뭐야?"
})

In [ ]:
# caching: 같은 질문을 계속 받으면 답변을 재사용해 비용을 절약할 수 있게 해준다.
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.globals import set_llm_cache, set_debug
from langchain.cache import InMemoryCache, SQLiteCache

set_llm_cache(SQLiteCache("cache.db"))
# set_llm_cache(InMemoryCache())
set_debug(True)
# 무슨 일을 하고 있는지에 대한 로그를 보여준다.

chat_model = ChatOpenAI(temperature=0.1)

chat_model.predict("토마토 스파게티는 어떻게 만들어?")

In [ ]:
chat_model.predict("토마토 스파게티는 어떻게 만들어?")

In [ ]:
# 지출하는 비용을 아는 방법
# 모델을 어떻게 저장하고 불러오는지에 대한 방법

from langchain.chat_models import ChatOpenAI
from langchain.callbacks import get_openai_callback

chat_model = ChatOpenAI(temperature=0.1)

with get_openai_callback() as test:
    a = chat_model.predict("소주 만드는 방법 알려줘")
    b = chat_model.predict("토마토 스파게티 만드는 법 알려줘")
    print(a, b, "\n")
    print(test)

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.llms.openai import OpenAI

chat = OpenAI(temperature=0.1, max_tokens=450, model="gpt-3.5-turbo-16k")

chat.save("model.json")

In [ ]:
from langchain.llms.loading import load_llm

chat = load_llm("model.json")
chat

In [ ]:
# langchain에는 여러 종류의 메모리가 있다. / 각자 저장 방식도 다르다
# 챗봇에 메모리를 추가하지 않으면 기억할 수 없다.
# 메모리 종류, 차이점 -> 메모리 탑재 방법


# 1. Conversation Buffer = 단순, 이전 대화 내용 전체를 저장 / 대화가 길어지면 메모리도 커져 비효율적이다.
# text completion 할 때 유용, 예측할 때, 텍스트를 자동완성하고 싶을 때 
from langchain.memory import ConversationBufferMemory

# memory = ConversationBufferMemory()
memory = ConversationBufferMemory(return_messages=True) # 챗모델 사용

memory.save_context({"input":"Hi"}, {"output": "How are you?"})

memory.load_memory_variables({})

In [ ]:
# 2. conversation Buffer Window Memory
# 대화 기록 최소화, 가장 오래된 메시지는 버린다. -> 모든 대화 내용을 저장하지 않아도 된다. / 최근 대화에만 집중한다.
# 메모리의 사이즈가 게속 늘어나지 않지만, 챗봇이 예전 대화를 기억하지 못 한다.

from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(
    return_messages=True,
    k = 4 # 버퍼 윈도우의 사이즈 = 몇 개의 메세지를 저장할 것인가
)

def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})

In [ ]:
add_message(1, 1)
add_message(2, 2)
add_message(3, 3)
add_message(4, 4)


In [ ]:
memory.load_memory_variables({})

In [ ]:
add_message(5, 5)

In [ ]:
memory.load_memory_variables({})

In [13]:
# 3. Conversation Summary Memory
# message를 그대로 저장하지 않고 요약을 자체적으로 해준다. -> 긴 conversation에서 효과적이다.
# 초반에는 Conversation Summary Memory는 이전보다 더 많은 토큰과 저장공간을 차지하지만 이후에 효과적

# 시간이 지남에 따라 대화의 요약 -> 시간이 지남에 따라 대화의 정보를 압축하는 데 유용
# 대화가 진행되는 대로 요약하고 현재 요약 내용을 메모리에 저장 

from langchain.memory import ConversationSummaryMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryMemory(llm=llm)

def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})
    
def get_history():
    return memory.load_memory_variables({})

add_message("안녕 나는 장태훈이야, 나는 한국에서 살아", "와 이거 정말 멋진데!!")

In [14]:
add_message("한국은 정말 멋져!!", "나도 갈 수 있었으면 좋겠다.")

In [15]:
get_history()

{'history': 'The human introduces themselves as Jang Taehoon from Korea. The AI responds enthusiastically to the introduction, expressing a desire to visit Korea as well.'}

In [8]:
# 4. Conversation Summary Buffer Memory
# 두 가지 아이디어 결합 -> 메모리에 최근 상호 작용의 버퍼 저장 + 오래된 상호 작용을 요약으로 컴파일 -> 가장 최근의 상호작용을 계속 추적
# 메모리에 보내온 메시지의 수를 저장, 

from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=150, return_messages=True)
# max_token_limit=10: 가능한 메시지 토큰 수의 최대값

def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})
    
def get_history():
    return memory.load_memory_variables({})

add_message("안녕 나는 장태훈이야, 나는 한국에서 살아", "와 이거 정말 멋진데!!")

In [9]:
get_history()

{'history': [SystemMessage(content='The human introduces themselves as Jang Taehoon from Korea. The AI responds enthusiastically to the introduction.')]}

In [10]:
add_message("한국은 정말 멋져!!", "나도 갈 수 있었으면 좋겠다.")

In [11]:
get_history()

{'history': [SystemMessage(content='The human introduces themselves as Jang Taehoon from Korea and expresses admiration for the country. The AI responds enthusiastically to the introduction and expresses a desire to visit Korea.')]}

In [12]:
add_message("한국이랑 아르헨나까지의 거리가 얼마나 되는지 알려 줄 수 있어?", "미안해!! 나도 잘 모르겠어 근데 엄청 멀꺼야 지구 반대퍈에 있거든!!")

In [13]:
get_history()

{'history': [SystemMessage(content='The human introduces themselves as Jang Taehoon from Korea and expresses admiration for the country. The AI responds enthusiastically to the introduction and expresses a desire to visit Korea, but admits not knowing the distance between Korea and Argentina, guessing it must be very far since they are on opposite sides of the Earth.')]}

In [14]:
add_message("곧 대한민국에서는 총선이야. 총선은 국회의원을 뽑는 선거인데 이번엔 누가 뽑힐지 잘 모르겠어 너는 혹시 알아? 알려줄 수 있어?", "미안해!! 나도 잘 모르겠어 근데 일 잘 하는 사람이 뽑힐 거야!")

In [15]:
get_history()

{'history': [SystemMessage(content='The human introduces themselves as Jang Taehoon from Korea and expresses admiration for the country. The AI responds enthusiastically to the introduction and expresses a desire to visit Korea, but admits not knowing the distance between Korea and Argentina, guessing it must be very far since they are on opposite sides of the Earth. Jang Taehoon mentions the upcoming general elections in Korea and asks if the AI knows who might be elected. The AI apologizes for not knowing but believes that competent individuals will be chosen.')]}

In [1]:
# 5. Conversation KG(knowledfe graph) Memory
# 대화 중의 엔티티의 knowledge graph를 만든다. -> 가장 중요한 것들만 뽑아내는 요약본

from langchain.memory import ConversationSummaryMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryMemory(llm=llm, return_messages=True)

def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})
    

add_message("안녕 나는 장태훈이야, 나는 한국에서 살아", "와 이거 정말 멋진데!!")

In [2]:
memory.load_memory_variables({"input": "장태훈이 누구인가요?"})

{'history': [SystemMessage(content='The human introduces themselves as Jang Taehoon from Korea. The AI responds enthusiastically to the introduction.')]}

In [3]:
add_message("장태훈은 고양이를 좋아해 그 중 쫀떡이를 제일 좋아해", "와 이거 정말 멋진데!!")

In [4]:
memory.load_memory_variables({"input": "장태훈이 좋아하는 건 뭐야?"})

{'history': [SystemMessage(content='The human introduces themselves as Jang Taehoon from Korea, mentioning their love for cats and specifically Jjondeok. The AI responds enthusiastically to the introduction, expressing admiration for their love of cats.')]}

In [14]:
# LLM chain
# off-the-shelf chain = 일반적인 목적을 가진 chain
# 스스로 무언가를 만들어 볼 때는 직접 커스텀해서 만든 chain을 활용하는 것이 좋다.

from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(temperature=0.1)

# 메모리는 template 안에 대화 기록을 저장하는 공간을 찾을 거고 자동으로 그 메모리의 기록을 chat_history에 넣는다.
# memory_key="chat_history" => 메모리를 위한 공간을 만들어 준다.
memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=120, memory_key="chat_history")

template = """
    너는 사람과 대화하며 도움을 주는 AI야
    {chat_history}
    Human:{question}
    You:
"""

chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=PromptTemplate.from_template(template),
    verbose=True
)

chain.predict(question="내 이름은 장태훈이야")



> Entering new LLMChain chain...
Prompt after formatting:

    너는 사람과 대화하며 도움을 주는 AI야
    
    Human:내 이름은 장태훈이야
    You:


> Finished chain.


'안녕하세요, 장태훈님! 만나서 반가워요. 무엇을 도와드릴까요?'

In [15]:
chain.predict(question="나는 수원에서 살아")



> Entering new LLMChain chain...
Prompt after formatting:

    너는 사람과 대화하며 도움을 주는 AI야
    Human: 내 이름은 장태훈이야
AI: 안녕하세요, 장태훈님! 만나서 반가워요. 무엇을 도와드릴까요?
    Human:나는 수원에서 살아
    You:


> Finished chain.


'수원은 경기도에 위치한 아름다운 도시죠. 수원에서 좋아하는 장소나 음식이 있나요? 무엇이든 물어보세요!'

In [16]:
chain.predict(question="내 이름이 뭐야")



> Entering new LLMChain chain...
Prompt after formatting:

    너는 사람과 대화하며 도움을 주는 AI야
    System: The human introduces themselves as 장태훈.
AI: 안녕하세요, 장태훈님! 만나서 반가워요. 무엇을 도와드릴까요?
Human: 나는 수원에서 살아
AI: 수원은 경기도에 위치한 아름다운 도시죠. 수원에서 좋아하는 장소나 음식이 있나요? 무엇이든 물어보세요!
    Human:내 이름이 뭐야
    You:


> Finished chain.


'AI: 죄송해요, 장태훈님. 제가 이름을 가지고 있지는 않아요. 하지만 저는 여러분의 질문에 답변해 드릴 수 있어요. 무엇이든 물어보세요!'

In [17]:
memory.load_memory_variables({})

{'chat_history': 'System: The human introduces themselves as 장태훈. The AI greets them and asks how it can help. The human mentions they live in Suwon. The AI comments on Suwon being a beautiful city in Gyeonggi-do and asks if there are any favorite places or foods in Suwon.\nHuman: 내 이름이 뭐야\nAI: AI: 죄송해요, 장태훈님. 제가 이름을 가지고 있지는 않아요. 하지만 저는 여러분의 질문에 답변해 드릴 수 있어요. 무엇이든 물어보세요!'}

In [18]:
# 대화 기반의 채팅으로 사용

from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=120, memory_key="chat_history", return_messages=True)
# 문자열로 바꾸지 말고 실제 메세지로 바꿔달라는 것을 의미

prompt = ChatPromptTemplate.from_messages([
    ("system", "너는 사람과 대화하며 도움을 주는 AI야"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{question}")
])
# ConversationBufferMemory로부터 요약본을 받아올 때 시스템 메시지도 추가된다.
# MessagesPlaceholder(variable_name="chat_history"): 메세지가 얼마나 많고 누구에게로부터 왔는지 모른다 -> memory class로 대체된다.

chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=prompt,
    verbose=True
)

chain.predict(question="내 이름은 장태훈이야")



> Entering new LLMChain chain...
Prompt after formatting:
System: 너는 사람과 대화하며 도움을 주는 AI야
Human: 내 이름은 장태훈이야

> Finished chain.


'안녕, 장태훈! 무엇을 도와드릴까요?'

In [19]:
chain.predict(question="나는 수원에서 살아")



> Entering new LLMChain chain...
Prompt after formatting:
System: 너는 사람과 대화하며 도움을 주는 AI야
Human: 내 이름은 장태훈이야
AI: 안녕, 장태훈! 무엇을 도와드릴까요?
Human: 나는 수원에서 살아

> Finished chain.


'수원은 정말 멋진 도시야! 수원에서 좋아하는 장소나 취미 활동이 있나요? 무엇인가 추천해 드릴까요?'

In [20]:
chain.predict(question="내 이름이 뭔지 알아?")



> Entering new LLMChain chain...
Prompt after formatting:
System: 너는 사람과 대화하며 도움을 주는 AI야
System: The human introduces themselves as 장태훈.
AI: 안녕, 장태훈! 무엇을 도와드릴까요?
Human: 나는 수원에서 살아
AI: 수원은 정말 멋진 도시야! 수원에서 좋아하는 장소나 취미 활동이 있나요? 무엇인가 추천해 드릴까요?
Human: 내 이름이 뭔지 알아?

> Finished chain.


'네, 당신의 이름은 장태훈이라고 말씀하셨죠. 어떻게 도와드릴까요, 장태훈님?'

In [22]:
# LCEL Based Memory

from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=120, memory_key="chat_history", return_messages=True)

prompt = ChatPromptTemplate.from_messages([
    ("system", "너는 사람과 대화하며 도움을 주는 AI야"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{question}")
])
# ConversationBufferMemory로부터 요약본을 받아올 때 시스템 메시지도 추가된다.
# MessagesPlaceholder(variable_name="chat_history"): 메세지가 얼마나 많고 누구에게로부터 왔는지 모른다 -> memory class로 대체된다.

def load_memory(input):
    return memory.load_memory_variables({})["chat_history"]

chain = RunnablePassthrough.assign(chat_history=load_memory) | prompt | llm

def invoke_chain(question):
    result = chain.invoke({
        # "chat_history": load_memory()
        "question": question
    })
    memory.save_context({"input": question}, {"output": result.content})
    print(result)

In [23]:
invoke_chain("내 이름은 장태훈이야")

content='안녕, 장태훈! 무엇을 도와드릴까요?'


In [24]:
invoke_chain("내 이름이 뭔지 알아?")

content='네, 당신의 이름은 장태훈이라고 말씀하셨죠. 어떻게 도와드릴까요?'
